In [1]:
!nvidia-smi


Tue Aug 19 18:22:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
%%writefile bfs.h
#pragma once
#include <cuda_runtime.h>
#include <stdio.h>

#define THREADS_PER_BLOCK 256

#define CHECK_CUDA_ERROR(call) do{cudaError_t e=(call);if(e!=cudaSuccess){fprintf(stderr,"CUDA error %s:%d: %s\n",__FILE__,__LINE__,cudaGetErrorString(e));exit(1);}}while(0)

void bfs_gpu(int source, int num_vertices, int num_edges, int* h_edges, int* h_dest, int* h_labels);


Overwriting bfs.h


In [7]:
%%writefile bfs_kernel.cu
#include "bfs.h"

__global__ void bfs_kernel(int level, int n, const int* edges, const int* dest, int* labels, int* done){
    int v = blockIdx.x * blockDim.x + threadIdx.x;
    if (v >= n) return;
    if (labels[v] != level) return;

    int s = edges[v];
    int e = edges[v + 1];
    for (int i = s; i < e; i++){
        int u = dest[i];
        if (atomicCAS(&labels[u], -1, level + 1) == -1){
            *done = 1;  // mark progress found
        }
    }
}

extern "C" void launch_bfs_kernel(int level, int num_vertices, const int* d_edges, const int* d_dest, int* d_labels, int* d_done){
    int threadsPerBlock = THREADS_PER_BLOCK;
    int blocksPerGrid = (num_vertices + threadsPerBlock - 1) / threadsPerBlock;
    bfs_kernel<<<blocksPerGrid, threadsPerBlock>>>(level, num_vertices, d_edges, d_dest, d_labels, d_done);
}


Overwriting bfs_kernel.cu


In [14]:
%%writefile bfs.cu
#include <cuda_runtime.h>
#include <vector>
#include <iostream>
#include <stdio.h>

#define TPB 256
#define CE(x) do{cudaError_t e=(x); if(e!=cudaSuccess){fprintf(stderr,"CUDA error %s:%d: %s\n",__FILE__,__LINE__,cudaGetErrorString(e)); exit(1);} }while(0)

__global__ void bfs_kernel(int level, int n, const int* edges, const int* dest, int* labels, int* changed){
    int v = blockIdx.x * blockDim.x + threadIdx.x;
    if (v >= n) return;
    if (labels[v] != level) return;
    int s = edges[v];
    int e = edges[v + 1];
    for (int i = s; i < e; i++){
        int u = dest[i];
        if (atomicCAS(&labels[u], -1, level + 1) == -1){
            atomicExch(changed, 1);
        }
    }
}

void bfs_gpu(int source, int n, int m, const int* h_edges, const int* h_dest, int* h_labels){
    int *d_edges, *d_dest, *d_labels, *d_changed;
    CE(cudaMalloc(&d_edges, (n + 1) * sizeof(int)));
    CE(cudaMalloc(&d_dest,  m * sizeof(int)));
    CE(cudaMalloc(&d_labels, n * sizeof(int)));
    CE(cudaMalloc(&d_changed, sizeof(int)));

    CE(cudaMemcpy(d_edges, h_edges, (n + 1) * sizeof(int), cudaMemcpyHostToDevice));
    CE(cudaMemcpy(d_dest,  h_dest,  m * sizeof(int),       cudaMemcpyHostToDevice));
    CE(cudaMemset(d_labels, 0xFF, n * sizeof(int)));

    int zero = 0;
    CE(cudaMemcpy(d_labels + source, &zero, sizeof(int), cudaMemcpyHostToDevice));

    int level = 0, h_changed;
    int blocks = (n + TPB - 1) / TPB;
    do{
        h_changed = 0;
        CE(cudaMemcpy(d_changed, &h_changed, sizeof(int), cudaMemcpyHostToDevice));

        bfs_kernel<<<blocks, TPB>>>(level, n, d_edges, d_dest, d_labels, d_changed);
        CE(cudaGetLastError());
        CE(cudaDeviceSynchronize());

        CE(cudaMemcpy(&h_changed, d_changed, sizeof(int), cudaMemcpyDeviceToHost));
        level++;
    } while (h_changed && level < n);

    CE(cudaMemcpy(h_labels, d_labels, n * sizeof(int), cudaMemcpyDeviceToHost));
    cudaFree(d_edges); cudaFree(d_dest); cudaFree(d_labels); cudaFree(d_changed);
}

int main(){
    int n = 6;
    // CSR representation
    std::vector<int> edges = {0,2,4,6,7,9,10};
    std::vector<int> dest  = {1,2, 0,3, 1,4, 4, 2,5, 3};
    std::vector<int> labels(n, -1);

    bfs_gpu(0, n, (int)dest.size(), edges.data(), dest.data(), labels.data());

    for(int i=0;i<n;i++) std::cout << labels[i] << (i+1==n?'\n':' ');
    return 0;
}


Overwriting bfs.cu


In [ ]:
!nvcc -O3 bfs.cu -o bfs
!./bfs
